In [1]:
from datasets import load_from_disk
dataset = load_from_disk("coyo-0.1m")

/home/sic/Documents/projects/kakao-keyword-to-prompt-generator/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

Dataset({
    features: ['id', 'url', 'text', 'width', 'height', 'image_phash', 'text_length', 'word_count', 'num_tokens_bert', 'num_tokens_gpt', 'num_faces', 'clip_similarity_vitb32', 'clip_similarity_vitl14', 'nsfw_score_opennsfw2', 'nsfw_score_gantman', 'watermark_score', 'aesthetic_score_laion_v2', 'keywords'],
    num_rows: 100000
})

In [3]:
dataset[0]

{'id': 2680060225205,
 'url': 'https://cdn.shopify.com/s/files/1/0286/3900/2698/products/TVN_Huile-olive-infuse-et-s-227x300_e9a90ffd-b6d2-4118-95a1-29a5c7a05a49_800x.jpg?v=1616684087',
 'text': 'Olive oil infused with Tuscany herbs',
 'width': 227,
 'height': 300,
 'image_phash': '9f91e133b1924e4e',
 'text_length': 36,
 'word_count': 6,
 'num_tokens_bert': 6,
 'num_tokens_gpt': 9,
 'num_faces': 0,
 'clip_similarity_vitb32': 0.19921875,
 'clip_similarity_vitl14': 0.147216796875,
 'nsfw_score_opennsfw2': 0.0058441162109375,
 'nsfw_score_gantman': 0.018961310386657715,
 'watermark_score': 0.11015450954437256,
 'aesthetic_score_laion_v2': 4.871710777282715,
 'keywords': ['tuscany herbs', 'olive oil', 'oil infused', 'tuscany', 'olive']}

In [4]:
concat_keywords = [', '.join(keyword) for keyword in dataset['keywords']]

In [5]:
len(concat_keywords)

100000

In [6]:
dataset = dataset.add_column("concat_keywords", concat_keywords)

In [7]:
dataset = dataset.train_test_split(test_size=0.1)

In [8]:
dataset['test'][0]

{'id': 1142461592569,
 'url': 'https://www.bcf.com.au/dw/image/v2/BBRV_PRD/on/demandware.static/-/Sites-srg-internal-master-catalog/default/dwebb77179/images/598598/BCF_598598_hi-res.jpg?sw=233&sh=233&sm=fit&q=90',
 'text': 'Wanderer Retro Stripe Camp Chair, , bcf_hi-res',
 'width': 233,
 'height': 233,
 'image_phash': 'b03093cbcd658ec7',
 'text_length': 46,
 'word_count': 7,
 'num_tokens_bert': 14,
 'num_tokens_gpt': 16,
 'num_faces': 0,
 'clip_similarity_vitb32': 0.349609375,
 'clip_similarity_vitl14': 0.2822265625,
 'nsfw_score_opennsfw2': 0.0006093978881835938,
 'nsfw_score_gantman': 0.021631531417369843,
 'watermark_score': 0.08680956065654755,
 'aesthetic_score_laion_v2': 4.798225402832031,
 'keywords': ['camp chair',
  'chair bcf_hi',
  'wanderer retro',
  'chair',
  'retro stripe'],
 'concat_keywords': 'camp chair, chair bcf_hi, wanderer retro, chair, retro stripe'}

In [9]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [10]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sic (buffett). Use `wandb login --relogin` to force relogin


True

In [11]:
model_name = "facebook/bart-base"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
encoder_max_length = 512
decoder_max_length = 512

In [13]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source_tokenized = tokenizer(
        batch["concat_keywords"], padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        batch["text"], padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch

tokenized_dataset = dataset.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=dataset["train"].column_names
)

Map: 100%|██████████| 10000/10000 [00:03<00:00, 2894.73 examples/s]


In [14]:
dataset["train"][0]

{'id': 5179731135344,
 'url': 'https://images1.westword.com/imager/u/745xauto/11636924/lee_clark_allen_crop.jpg',
 'text': 'Lee Clark Allen plays the Broadway Roxy on Friday, February 14.',
 'width': 745,
 'height': 584,
 'image_phash': 'f0bc189bc186cfe1',
 'text_length': 63,
 'word_count': 11,
 'num_tokens_bert': 13,
 'num_tokens_gpt': 14,
 'num_faces': 1,
 'clip_similarity_vitb32': 0.26513671875,
 'clip_similarity_vitl14': 0.24267578125,
 'nsfw_score_opennsfw2': 0.0144500732421875,
 'nsfw_score_gantman': 0.005059111397713423,
 'watermark_score': 0.005766104441136122,
 'aesthetic_score_laion_v2': 5.2899980545043945,
 'keywords': ['broadway roxy',
  'the broadway',
  'broadway',
  'clark allen',
  'allen plays'],
 'concat_keywords': 'broadway roxy, the broadway, broadway, clark allen, allen plays'}

In [15]:
metric = datasets.load_metric("rouge")

/tmp/ipykernel_145860/3269952567.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("rouge")


In [16]:
nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [17]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=1,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [18]:
wandb_run = wandb.init(
    project="bart prompt generator",
    config={
        "per_device_train_batch_size": training_args.per_device_train_batch_size,
        "learning_rate": training_args.learning_rate,
        "dataset": "coyo-0.1m",
    },
)

now = datetime.now()
current_time = now.strftime("%H%M%S")
wandb_run.name = "run_" + current_time

In [20]:
trainer.evaluate()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 8.240460395812988,
 'eval_rouge1': 58.3816,
 'eval_rouge2': 40.841,
 'eval_rougeL': 45.8618,
 'eval_rougeLsum': 46.1665,
 'eval_gen_len': 17.18,
 'eval_runtime': 312.4013,
 'eval_samples_per_second': 32.01,
 'eval_steps_per_second': 2.001}

In [19]:
trainer.train()

/home/sic/Documents/projects/kakao-keyword-to-prompt-generator/venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,6.672400
100,5.455900
150,5.128700
200,4.983100
250,4.907100
300,4.956100
350,4.818600
400,4.780000
450,4.715200
500,4.761300


TrainOutput(global_step=5625, training_loss=4.342900630696614, metrics={'train_runtime': 3133.6653, 'train_samples_per_second': 28.72, 'train_steps_per_second': 1.795, 'total_flos': 2.74381406208e+16, 'train_loss': 4.342900630696614, 'epoch': 1.0})

In [20]:
trainer.evaluate()

{'eval_loss': 3.926250457763672,
 'eval_rouge1': 69.8846,
 'eval_rouge2': 57.3013,
 'eval_rougeL': 65.8986,
 'eval_rougeLsum': 66.0672,
 'eval_gen_len': 11.4394,
 'eval_runtime': 301.0498,
 'eval_samples_per_second': 33.217,
 'eval_steps_per_second': 2.076,
 'epoch': 1.0}

In [21]:
keywords = "cat, clean, dog, dirty"
inputs = tokenizer(
    keywords,
    padding="max_length",
    truncation=True,
    max_length=encoder_max_length,
    return_tensors="pt",
)
input_ids = inputs.input_ids.to(model.device)
attention_mask = inputs.attention_mask.to(model.device)
outputs = model.generate(input_ids, attention_mask=attention_mask)
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output_str)

['Cat and dog are dirty']


/home/sic/Documents/projects/kakao-keyword-to-prompt-generator/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [27]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["concat_keywords"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = dataset["test"].select(range(16))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

In [28]:
test_samples[0]

{'id': 1142461592569,
 'url': 'https://www.bcf.com.au/dw/image/v2/BBRV_PRD/on/demandware.static/-/Sites-srg-internal-master-catalog/default/dwebb77179/images/598598/BCF_598598_hi-res.jpg?sw=233&sh=233&sm=fit&q=90',
 'text': 'Wanderer Retro Stripe Camp Chair, , bcf_hi-res',
 'width': 233,
 'height': 233,
 'image_phash': 'b03093cbcd658ec7',
 'text_length': 46,
 'word_count': 7,
 'num_tokens_bert': 14,
 'num_tokens_gpt': 16,
 'num_faces': 0,
 'clip_similarity_vitb32': 0.349609375,
 'clip_similarity_vitl14': 0.2822265625,
 'nsfw_score_opennsfw2': 0.0006093978881835938,
 'nsfw_score_gantman': 0.021631531417369843,
 'watermark_score': 0.08680956065654755,
 'aesthetic_score_laion_v2': 4.798225402832031,
 'keywords': ['camp chair',
  'chair bcf_hi',
  'wanderer retro',
  'chair',
  'retro stripe'],
 'concat_keywords': 'camp chair, chair bcf_hi, wanderer retro, chair, retro stripe'}

In [26]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Generated Text", "Keywords"],
    )
)
print("\nTarget text:\n")
print(
    tabulate(list(enumerate(test_samples["text"])), headers=["Id", "Target text"])
)
print("\nSource documents:\n")
print(tabulate(list(enumerate(test_samples["concat_keywords"])), headers=["Id", "Original text"]))

  Id  Generated Text                                               Keywords
----  -----------------------------------------------------------  -------------------------------------------------------------------------
   0  W wanderer Retro Stripe Camp Chair Bcf_HI                    camp chair, chair bcf_hi, wanderer retro, chair, retro stripe
   1  TSMC Ultraviolet Lithography was developed in the 1970s.     ultraviolet lithography, tsmc ultraviolet, lithographic, lithography was,
   2  STP PoE Field termination plug for Cat 6A                    stp poe, termination plug, shielded straight, poe field, cat 6
   3  Smile from above extended mix                                smile from, extended mix, above extended, smile, from above
   4  friendly holidays at crossways house in garden flat          friendly holidays, garden flat, crossways house, holidays at, holidays
   5  friends of type ampersand                                    type ampersand, ampersands, friends of, friends
   

In [30]:
trainer.save_model("bart-coco")